- 84406개의 데이터
- ID : 샘플 별 고유 id
- 월 : 사건 발생월
- 요일 : 월요일 ~ 일요일
- 시간 : 사건 발생 시각
- 소관경찰서 : 사건 발생 구역의 담당 경찰서
- 소관지역 : 사건 발생 구역
- 사건발생거리 : 가장 가까운 경찰서에서 사건 현장까지의 거리
- 강수량(mm) 
- 강설량(mm)
- 적설량(cm) 
- 풍향 : 범죄발생지에서 바람이 부는 방향(최대 360도)
- 안개 : 가시거리가 1km 미만인 경우
- 짙은안개 : 가시거리가 200m 미만인 경우
- 번개
- 진눈깨비
- 서리
- 연기/연무 : 먼지, 연기가 하늘을 가리는 현상
- 눈날림
- 범죄발생지 : 범죄가 발생한 장소
- TARGET : 범죄타입 [0 : 강도, 1: 절도, 2: 상해]

In [1]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.preprocessing import *
from sklearn.feature_selection import *
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from scipy.special import softmax
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score

In [2]:
np.random.seed(42)

In [12]:
df = pd.read_csv('./train.csv', encoding='UTF-8')

In [13]:
df.head(3)

,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,TRAIN_00000,9,화요일,10,137,8.0,2.611124,0.0,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,TRAIN_00001,11,화요일,6,438,13.0,3.209093,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,TRAIN_00002,8,일요일,6,1729,47.0,1.619597,0.0,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1


In [14]:
df.describe()

,월,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,TARGET
count,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000,84406.000000
mean,6.430195,6.769507,1060.027581,26.881726,1.912424,24.608776,2.284407,23.430503,186.926107,0.385423,0.017842,0.144042,0.020330,0.010260,0.210755,0.008921,0.835355
std,3.108302,3.566390,698.380485,13.870968,0.958556,62.711211,15.852881,85.199896,98.299485,0.486698,0.132379,0.351134,0.141128,0.100771,0.407847,0.094030,0.819762
min,1.000000,1.000000,26.000000,5.000000,0.012269,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,4.000000,526.000000,13.000000,1.209985,0.000000,0.000000,0.000000,95.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.000000,7.000000,937.000000,27.000000,1.822279,0.625000,0.000000,0.000000,205.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,9.000000,10.000000,1638.000000,38.000000,2.476528,18.571429,0.000000,0.000000,260.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,12.000000,12.000000,2450.000000,54.000000,4.998936,614.875000,295.000000,649.800000,360.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [15]:
encoder = LabelEncoder()
df['요일'] = encoder.fit_transform(df[['요일']])
encoder = LabelEncoder()
df['범죄발생지'] = encoder.fit_transform(df[['범죄발생지']])

D:\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [16]:
df.head(3)

,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,TRAIN_00000,9,6,10,137,8.0,2.611124,0.0,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,2
1,TRAIN_00001,11,6,6,438,13.0,3.209093,0.0,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,0
2,TRAIN_00002,8,4,6,1729,47.0,1.619597,0.0,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,6,1


In [393]:
trainx, testx, trainy, testy = train_test_split(df.iloc[:,1:-1], df['TARGET'], random_state=42)

In [30]:
%%time
rf = RandomForestClassifier(criterion='gini', random_state=42)
rf.fit(trainx, trainy)
print(rf.score(testx, testy))
pred = rf.predict(testx)
print(f1_score(testy, pred, average='macro'))

0.527580324139892
0.4979727264775253
CPU times: total: 14.7 s
Wall time: 14.7 s


In [31]:
%%time
rf = RandomForestClassifier(criterion='entropy', random_state=42)
rf.fit(trainx, trainy)
print(rf.score(testx, testy))
pred = rf.predict(testx)
print(f1_score(testy, pred, average='macro'))

0.5255899914700028
0.49497460687209216
CPU times: total: 17.4 s
Wall time: 17.4 s


In [13]:
%%time
knc = KNeighborsClassifier(n_neighbors=20, weights='uniform')
knc.fit(trainx, trainy)
print(knc.score(testx, testy))
pred =knc.predict(testx)
print(f1_score(testy, pred, average='macro'))

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.446497962278457
0.3875223306124567
CPU times: total: 1min 31s
Wall time: 1min 2s


In [14]:
%%time
knc = KNeighborsClassifier(n_neighbors=20, weights='distance')
knc.fit(trainx, trainy)
print(knc.score(testx, testy))
pred =knc.predict(testx)
print(f1_score(testy, pred, average='macro'))

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.4404795753956971
0.4041979671620212
CPU times: total: 1min 28s
Wall time: 59.5 s


In [17]:
%%time
# scaler = MinMaxScaler()
# df[['월','요일','소관경찰서','소관지역','사건발생거리', '범죄발생지']] = scaler.fit_transform(df[['월','요일','소관경찰서','소관지역','사건발생거리', '범죄발생지']])
trainx, testx, trainy, testy = train_test_split(df.iloc[:,1:-1], df['TARGET'], random_state=42)

lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(trainx, trainy)
print(lr.score(testx, testy))
pred = lr.predict(testx)
# proba = lr.predict_proba(testx)
# decision = lr.decision_function(testx)
# proba = softmax(decision, axis=1)
print(f1_score(testy, pred, average='macro'))

0.4440337408776419
0.33432676270259587
CPU times: total: 1min 54s
Wall time: 30.8 s


D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [21]:
%%time
# scaler = MinMaxScaler()
# df[['월','요일','소관경찰서','소관지역','사건발생거리', '범죄발생지']] = scaler.fit_transform(df[['월','요일','소관경찰서','소관지역','사건발생거리', '범죄발생지']])
trainx, testx, trainy, testy = train_test_split(df.iloc[:,1:-1], df['TARGET'], random_state=42)

lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(trainx, trainy)
print(lr.score(testx, testy))
pred = lr.predict(testx)
# proba = lr.predict_proba(testx)
# decision = lr.decision_function(testx)
# proba = softmax(decision, axis=1)
print(f1_score(testy, pred, average='macro'))

0.4440337408776419
0.33432676270259587
CPU times: total: 1min 55s
Wall time: 32.8 s


D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
%%time
# scaler = MinMaxScaler()
# df[['월','요일','소관경찰서','소관지역','사건발생거리', '범죄발생지']] = scaler.fit_transform(df[['월','요일','소관경찰서','소관지역','사건발생거리', '범죄발생지']])
trainx, testx, trainy, testy = train_test_split(df.iloc[:,1:-1], df['TARGET'], random_state=42)

lr = LogisticRegression(C=1, max_iter=2000)
lr.fit(trainx, trainy)
print(lr.score(testx, testy))
pred = lr.predict(testx)
# proba = lr.predict_proba(testx)
# decision = lr.decision_function(testx)
# proba = softmax(decision, axis=1)
print(f1_score(testy, pred, average='macro'))

0.4615202350488105
0.37020443990136287
CPU times: total: 3min 43s
Wall time: 58.6 s


D:\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [377]:
class SoftmaxClassification(torch.nn.Module):
    def __init__(self, input_size):
        super(SoftmaxClassification, self).__init__()
        self.input_size = input_size
        self.linear1 = torch.nn.Linear(self.input_size, self.input_size, bias=True)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(self.input_size, 3, bias=True)
        self.softmax = torch.nn.Softmax()
    
    def forward(self, input_tensor):
        linear1 = self.linear1(input_tensor)
        linear1 = self.relu(linear1)
        linear2 = self.linear2(linear1)
        output = self.softmax(linear2)
        return output
    
x_train = torch.FloatTensor(np.array(trainx))
ytrain = torch.LongTensor(np.array(trainy))
y_train = torch.zeros(len(ytrain), 3)
y_train.scatter_(1, ytrain.unsqueeze(1), 1)

x_test = torch.FloatTensor(np.array(testx))
ytest = torch.LongTensor(np.array(testy))
y_test = torch.zeros(len(ytest), 3)
y_test.scatter_(1, ytest.unsqueeze(1), 1)

sm = SoftmaxClassification(len(x_train[0]))
epochs = 50
optimizer = torch.optim.SGD(sm.parameters(), lr=0.01)

for epoch in range(epochs+1):
    sm.train()
    optimizer.zero_grad()
    train_output = sm(x_train)
    train_loss = F.cross_entropy(train_output.squeeze(), y_train)
    train_loss.backward()
    optimizer.step()

In [381]:
x_train = torch.FloatTensor(np.array(trainx))
ytrain = torch.LongTensor(np.array(trainy))
y_train = torch.zeros(len(ytrain), 3)
y_train.scatter_(1, ytrain.unsqueeze(1), 1)

x_test = torch.FloatTensor(np.array(testx))
ytest = torch.LongTensor(np.array(testy))
y_test = torch.zeros(len(ytest), 3)
y_test.scatter_(1, ytest.unsqueeze(1), 1)

sm = SoftmaxClassification(len(x_train[0]))
epochs = 50
optimizer = torch.optim.SGD(sm.parameters(), lr=0.01)

sm.eval()
output = sm(x_test)
pred = []
for i in range(len(output)):
    _ = int((output[i]==max(output[i])).nonzero(as_tuple=True)[0])
    pred.append(_)
    
print(f1_score(testy, pred, average='macro'))

C:\Users\ray67\AppData\Local\Temp\ipykernel_7284\78874208.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(linear2)


0.17280541644967298


In [15]:
df_ = df.copy()
df_.drop(columns=['강수량(mm)', '강설량(mm)', '적설량(cm)', '풍향'], inplace=True)

In [395]:
trainx, testx, trainy, testy = train_test_split(df_.iloc[:,1:-1], df_['TARGET'], random_state=42)

In [18]:
%%time
rf = RandomForestClassifier(criterion='gini', random_state=42)
rf.fit(trainx, trainy)
print(rf.score(testx, testy))
pred = rf.predict(testx)
print(f1_score(testy, pred, average='macro'))

<timed exec>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.5218936593687802
0.49135115791302236
CPU times: total: 11.1 s
Wall time: 11.1 s


In [19]:
%%time
rf = RandomForestClassifier(criterion='entropy', random_state=42)
rf.fit(trainx, trainy)
print(rf.score(testx, testy))
pred = rf.predict(testx)
print(f1_score(testy, pred, average='macro'))

<timed exec>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


0.5245474362619656
0.4935094304771912
CPU times: total: 14 s
Wall time: 14 s


In [20]:
%%time
knc = KNeighborsClassifier(n_neighbors=20, weights='uniform')
knc.fit(trainx, trainy)
print(knc.score(testx, testy))
pred =knc.predict(testx)
print(f1_score(testy, pred, average='macro'))

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.5043123874514264
0.4574422480451366
CPU times: total: 4.14 s
Wall time: 4.15 s


In [21]:
%%time
knc = KNeighborsClassifier(n_neighbors=20, weights='distance')
knc.fit(trainx, trainy)
print(knc.score(testx, testy))
pred =knc.predict(testx)
print(f1_score(testy, pred, average='macro'))

D:\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.505070609420908
0.46859212714997894
CPU times: total: 2.98 s
Wall time: 2.96 s


In [402]:
%%time
%%time
scaler = MinMaxScaler()
df_[['월','요일','소관경찰서','소관지역','사건발생거리', '범죄발생지']] = scaler.fit_transform(df_[['월','요일','소관경찰서','소관지역','사건발생거리', '범죄발생지']])
trainx, testx, trainy, testy = train_test_split(df_.iloc[:,1:-1], df_['TARGET'], random_state=42)

lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(trainx, trainy)
print(lr.score(testx, testy))
proba = lr.predict_proba(testx)
# decision = lr.decision_function(testx)
# proba = softmax(decision, axis=1)
print(f1_score(testy, pred, average='macro'))
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(trainx, trainy)
print(lr.score(testx, testy))
proba = lr.predict_proba(testx)
# decision = lr.decision_function(testx)
# proba = softmax(decision, axis=1)
print(f1_score(testy, pred, average='macro'))

0.4589138470287177
0.15508442731250835
0.4589138470287177
0.15508442731250835
CPU times: total: 37 s
Wall time: 9.28 s
CPU times: total: 37 s
Wall time: 9.28 s


In [385]:
class SoftmaxClassification(torch.nn.Module):
    def __init__(self, input_size):
        super(SoftmaxClassification, self).__init__()
        self.input_size = input_size
        self.linear1 = torch.nn.Linear(self.input_size, self.input_size, bias=True)
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(self.input_size, 3, bias=True)
        self.softmax = torch.nn.Softmax()
    
    def forward(self, input_tensor):
        linear1 = self.linear1(input_tensor)
        linear1 = self.relu(linear1)
        linear2 = self.linear2(linear1)
        output = self.softmax(linear2)
        return output
    
x_train = torch.FloatTensor(np.array(trainx))
ytrain = torch.LongTensor(np.array(trainy))
y_train = torch.zeros(len(ytrain), 3)
y_train.scatter_(1, ytrain.unsqueeze(1), 1)

x_test = torch.FloatTensor(np.array(testx))
ytest = torch.LongTensor(np.array(testy))
y_test = torch.zeros(len(ytest), 3)
y_test.scatter_(1, ytest.unsqueeze(1), 1)

sm = SoftmaxClassification(len(x_train[0]))
epochs = 50
optimizer = torch.optim.SGD(sm.parameters(), lr=0.01)

for epoch in range(epochs+1):
    sm.train()
    optimizer.zero_grad()
    train_output = sm(x_train)
    train_loss = F.cross_entropy(train_output.squeeze(), y_train)
    train_loss.backward()
    optimizer.step()
    
sm.eval()
output = sm(x_test)
pred = []
for i in range(len(output)):
    _ = int((output[i]==max(output[i])).nonzero(as_tuple=True)[0])
    pred.append(_)
    
print(f1_score(testy, pred, average='macro'))

C:\Users\ray67\AppData\Local\Temp\ipykernel_7284\2042646859.py:14: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(linear2)


0.15508442731250835
